In [1]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
from google.colab import drive
from sklearn.model_selection import train_test_split

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Set the path of the dataset folder in Google Drive
path = "/content/drive/MyDrive/eye_set"

In [4]:
# Define the classes as the directory names
classes = sorted(os.listdir(path))

In [5]:
# Create a dictionary that maps class names to integer labels
class_dict = {class_name: i for i, class_name in enumerate(classes)}

In [ ]:
# Load the image files and labels
data = []
labels = []
for class_name in classes:
    class_path = os.path.join(path, class_name)
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (224, 224))
        data.append(image)
        labels.append(class_dict[class_name])

In [7]:
# Convert the data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

In [8]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [9]:
# Normalize the image data
X_train = X_train / 255.0
X_val = X_val / 255.0

In [10]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(len(classes), activation="softmax")
])

In [11]:
# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [12]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

Epoch 1/10
19/19 [==============================] - 16s 132ms/step - loss: 1.8412 - accuracy: 0.2600 - val_loss: 1.3021 - val_accuracy: 0.4267
Epoch 2/10
19/19 [==============================] - 1s 59ms/step - loss: 1.3027 - accuracy: 0.4333 - val_loss: 1.1772 - val_accuracy: 0.5200
Epoch 3/10
19/19 [==============================] - 1s 59ms/step - loss: 1.2417 - accuracy: 0.4783 - val_loss: 1.1833 - val_accuracy: 0.5733
Epoch 4/10
19/19 [==============================] - 1s 53ms/step - loss: 1.1862 - accuracy: 0.5333 - val_loss: 1.1325 - val_accuracy: 0.5467
Epoch 5/10
19/19 [==============================] - 1s 55ms/step - loss: 1.0960 - accuracy: 0.5300 - val_loss: 1.1755 - val_accuracy: 0.5733
Epoch 6/10
19/19 [==============================] - 1s 53ms/step - loss: 1.0712 - accuracy: 0.5817 - val_loss: 1.1315 - val_accuracy: 0.5600
Epoch 7/10
19/19 [==============================] - 1s 53ms/step - loss: 0.9865 - accuracy: 0.6000 - val_loss: 1.1241 - val_accuracy: 0.4733
Epoch 8/10


In [13]:
# Evaluate the model on the test set
X_test, y_test = [], []
for class_name in classes:
    class_path = os.path.join(path, class_name)
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (224, 224))
        X_test.append(image)
        y_test.append(class_dict[class_name])
X_test = np.array(X_test) / 255.0
y_test = np.array(y_test)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print("Test accuracy:", test_acc)

24/24 - 1s - loss: 0.9741 - accuracy: 0.6413 - 723ms/epoch - 30ms/step
Test accuracy: 0.6413333415985107


In [14]:
# Save the model
model.save(os.path.join(path, "model.h5"))

In [22]:
# Load a test image
image = cv2.imread('/content/drive/MyDrive/eye_set/Severe DR/Severe DR_10.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))

# Convert the image to a numpy array and normalize the pixel values
image = np.array(image) / 255.0

# Reshape the image to match the expected shape of the model's input layer
image = np.reshape(image, (1, 224, 224, 3))

# Make a prediction on the image
y_pred = model.predict(image)

# Print the predicted class label
predicted_class = np.argmax(y_pred)
print(f"Predicted class: {classes[predicted_class]}")

# Evaluate the accuracy of the model on the test set
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Test accuracy: {test_acc}")

1/1 [==============================] - 0s 19ms/step
Predicted class: Severe DR
5/5 [==============================] - 0s 23ms/step - loss: 1.1268 - accuracy: 0.5333
Test accuracy: 0.5333333611488342
